# 🖼️ 1. Datensatz vorbereiten

Jede Gruppe erhält einen Ordner mit **Rohbildern**. Diese sollen in [Makesense.ai](https://www.makesense.ai) annotiert werden.

## 🔧 Schritt 1: Annotation mit Makesense.ai
0. Ladet die Rohbilder mit eurer Gruppennummer unter folgender [URL](https://databox.bonn.de/public/upload-shares/YhXlVGe57thQAg8WUjVlZFNK6z4LORiq) runter
1. Öffnet [https://www.makesense.ai](https://www.makesense.ai)
2. Bilder hochladen
3. Modus: **Segmentation**
4. Klasse: z. B. `baum`, `überdachung`
5. Export: Format `Pascal VOC Segmentation (PNG masks)`

👉 Jetzt bitte eure annotierten Masken und Bilder wieder hochladen. Verwende folgenden Platzhalter:


In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import os
from IPython.display import display

# Gruppennummer eintragen
gruppen_nummer = 2

# Arbeitsverzeichnis dynamisch zusammensetzen
annotations_file = f"datensatz_gruppe{gruppen_nummer}/annotations.json"
images_dir = f"datensatz_gruppe{gruppen_nummer}"

# COCO laden
coco = COCO(annotations_file)
image_ids = coco.getImgIds()

# Nimm das erste Bild
img_id = image_ids[0]
img_info = coco.loadImgs(img_id)[0]
img_path = os.path.join(images_dir, img_info['file_name'])

# Bild laden
image = Image.open(img_path)

# Plot vorbereiten
fig = plt.figure(figsize=(8, 8))
plt.imshow(image)
ax = plt.gca()

# Annotations laden
ann_ids = coco.getAnnIds(imgIds=img_id)
anns = coco.loadAnns(ann_ids)

# Maske zeichnen
for ann in anns:
    if 'segmentation' in ann and isinstance(ann['segmentation'], list):
        for seg in ann['segmentation']:
            poly = np.array(seg).reshape((len(seg) // 2, 2))
            patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(patch)

plt.title(img_info['file_name'])
plt.axis('off')

## 📈 Datensatz-Erweiterung durch Augmentation

Um die Robustheit und Generalisierungsfähigkeit des Modells zu verbessern, wurde der Datensatz durch gezielte Bildaugmentierung erweitert. Dabei wurde zu jedem bestehenden Bild **eine sinnvolle Transformation** erzeugt.

Die angewendeten Augmentierungsschritte umfassen:
- **Horizontales Spiegeln**
- **Leichte Rotation (±15°)**
- **Änderung von Helligkeit und Kontrast**

Für jedes Originalbild wurde ein neues Bild generiert. Die zugehörigen Segmentierungs-Masken wurden dabei korrekt mittransformiert. Die augmentierten Bilder und Masken wurden zusammen mit den Originalen in einer neuen COCO-kompatib


In [ ]:
import albumentations as A
import cv2
import json
import os
import numpy as np
import shutil
from pycocotools import mask as mask_utils
import uuid
from tqdm import tqdm
import random

# 📂 Verzeichnisse
source_dir = images_dir
target_dir = f"datensatz_gruppe{gruppen_nummer}_augmented"
os.makedirs(target_dir, exist_ok=True)

# 📖 Originaldaten laden
with open(os.path.join(source_dir, "annotations.json"), "r") as f:
    coco_data = json.load(f)

# 🧱 Neue COCO-Struktur → beginne mit Kopie der Originale
extended_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data["categories"]
}

# Kopiere originale Bilder & übernehme JSON-Einträge
for img_entry in coco_data["images"]:
    src_path = os.path.join(source_dir, img_entry["file_name"])
    dst_path = os.path.join(target_dir, img_entry["file_name"])
    if not os.path.exists(dst_path):
        shutil.copy(src_path, dst_path)
    extended_data["images"].append(img_entry)

for ann in coco_data["annotations"]:
    extended_data["annotations"].append(ann)

# ID-Offsets
next_image_id = max(img["id"] for img in coco_data["images"]) + 1
next_ann_id = max(ann["id"] for ann in coco_data["annotations"]) + 1

# Bild-ID → Annotations
img_to_anns = {}
for ann in coco_data["annotations"]:
    img_to_anns.setdefault(ann["image_id"], []).append(ann)

# ✅ Sichere Augmentierungen
aug_choices = [
    ("flipH", A.HorizontalFlip(p=1.0)),
    ("flipV", A.VerticalFlip(p=1.0)),
    ("rot90", A.RandomRotate90(p=1.0))
]

# Augmentieren
for img_entry in tqdm(coco_data["images"], desc="Augmentiere Bilder"):
    file_name = img_entry["file_name"]
    img_path = os.path.join(source_dir, file_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    anns = img_to_anns.get(img_entry["id"], [])
    if not anns:
        continue

    masks = []
    for ann in anns:
        for seg in ann["segmentation"]:
            pts = np.array(seg).reshape(-1, 2)
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [pts.astype(np.int32)], 1)
            masks.append(mask)

    aug_name, transform = random.choice(aug_choices)
    transformed = transform(image=img, masks=masks)
    aug_img = transformed["image"]
    aug_masks = transformed["masks"]

    new_filename = f"{aug_name}_{file_name}"
    new_img_path = os.path.join(target_dir, new_filename)
    cv2.imwrite(new_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

    extended_data["images"].append({
        "id": next_image_id,
        "file_name": new_filename,
        "width": aug_img.shape[1],
        "height": aug_img.shape[0]
    })

    for mask in aug_masks:
        rle = mask_utils.encode(np.asfortranarray(mask.astype(np.uint8)))
        area = mask_utils.area(rle).item()
        bbox = mask_utils.toBbox(rle).tolist()

        seg = mask_utils.encode(np.asfortranarray(mask))
        seg["counts"] = seg["counts"].decode("utf-8")

        extended_data["annotations"].append({
            "id": next_ann_id,
            "image_id": next_image_id,
            "category_id": 1,
            "segmentation": seg,
            "area": area,
            "bbox": bbox,
            "iscrowd": 0
        })
        next_ann_id += 1

    next_image_id += 1

# Speichern
json_out = os.path.join(target_dir, "annotations_augmented.json")
with open(json_out, "w") as f:
    json.dump(extended_data, f)

print(f"✅ Erweiterung abgeschlossen. Gesamt-Datensatz unter: {target_dir}, JSON: {json_out}")


## Visualisierung der Augmentation

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import os
from IPython.display import display
from pycocotools import mask as mask_utils

# Pfade
json_path = f"{target_dir}/annotations_augmented.json"
img_dir = target_dir

# COCO laden
coco = COCO(json_path)
images = coco.dataset["images"]

# 📌 Nur augmentierte Bilder auswählen (z. B. "flipH_", "flipV_", "rot90_")
aug_images = [img for img in images if img["file_name"].startswith(("flip", "rot", "flipH", "flipV"))]

# Erstes augmentiertes Bild
img_info = aug_images[0]
img_path = os.path.join(img_dir, img_info["file_name"])
image = Image.open(img_path)

# Masken laden
ann_ids = coco.getAnnIds(imgIds=img_info["id"])
anns = coco.loadAnns(ann_ids)

# Plot
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(image)

for ann in anns:
    seg = ann["segmentation"]
    
    if isinstance(seg, list):  # Polygon
        for s in seg:
            poly = np.array(s).reshape((len(s) // 2, 2))
            patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(patch)
    elif isinstance(seg, dict) and "counts" in seg:  # RLE
        rle = {
            "counts": seg["counts"].encode("utf-8"),
            "size": seg["size"]
        }
        mask = mask_utils.decode(rle)
        ax.contour(mask, colors='red', linewidths=2)

ax.set_title(f"Augmentiertes Bild: {img_info['file_name']}")
ax.axis("off")
display(fig)
plt.close(fig)

## Datensatz prüfen

In [ ]:
import os
import json


# 📖 JSON laden
with open(json_path, "r") as f:
    data = json.load(f)

# 🔎 Bildnamen aus JSON
json_images = {img["file_name"] for img in data["images"]}
json_image_ids = {img["id"] for img in data["images"]}

# 🔎 Bilddateien im Ordner
folder_images = {f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))}

# 🔎 IDs aus Annotations
referenced_ids = {ann["image_id"] for ann in data["annotations"]}

# ✅ Checks
missing_files = json_images - folder_images
unreferenced_files = folder_images - json_images
annotations_without_images = referenced_ids - json_image_ids

# 🖨️ Ergebnis
print("📁 Bilder in JSON, aber nicht im Ordner:", missing_files if missing_files else "✅ Keine")
print("📁 Bilder im Ordner, aber nicht in JSON:", unreferenced_files if unreferenced_files else "✅ Keine")
print("📛 Annotations mit fehlenden Bild-IDs:", annotations_without_images if annotations_without_images else "✅ Keine")


## Datensatz Hochladen

In [ ]:
import os
import requests
from getpass import getpass
from tqdm import tqdm


target_dir = f"datensatz_gruppe{gruppen_nummer}_augmented"

# 🌐 WebDAV-Ziel ohne abschließenden Slash!
webdav_base = f"https://mrzinken.duckdns.org/remote.php/dav/files/hackathon2025/HackathonBonn/gruppe{gruppen_nummer}"

# 🔐 Zugangsdaten
username = "hackathon2025"
password = getpass("🔑 Passwort eingeben:")

# 📤 Upload-Schleife
for fname in tqdm(os.listdir(target_dir), desc="📤 Upload läuft"):
    fpath = os.path.join(target_dir, fname)
    if os.path.isfile(fpath):
        remote_url = f"{webdav_base}/{fname}"
        with open(fpath, "rb") as f:
            r = requests.put(remote_url, data=f, auth=(username, password))
        if r.status_code in [200, 201, 204]:
            print(f"✅ Hochgeladen: {fname}")
        else:
            print(f"❌ Fehler bei {fname}: {r.status_code} {r.text}")